In [2]:
!pip install tensorflow

In [3]:
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fou

In [4]:
!pip install tensorflow_hub

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
from keras.layers import Dense,Dropout

In [7]:
df=pd.read_csv('train.csv', encoding='latin1')
df=df[:5000]

In [8]:
df=df.dropna()

In [9]:
df['sentiment'].value_counts()

neutral     2022
positive    1592
negative    1385
Name: sentiment, dtype: int64

In [10]:
df=df[['text','sentiment']]

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") ##converts raw text to numeric input tensors for encoder
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4") ##takes input sequence and creates contextual representation

In [12]:
df['Labels'] = df['sentiment'].map({'neutral': 0, 'negative':-1,'positive':1}) ##mapping labels

In [13]:
df

,text,sentiment,Labels
0,"I`d have responded, if I were going",neutral,0
1,Sooo SAD I will miss you here in San Diego!!!,negative,-1
2,my boss is bullying me...,negative,-1
3,what interview! leave me alone,negative,-1
4,"Sons of ****, why couldn`t they put them on t...",negative,-1
...,...,...,...
4995,look like i can only quick post from the dash...,neutral,0
4996,Thanks Elaine. I`m clipping reviews for her s...,positive,1
4997,"K now that, that fiasco is all wrapped and goo...",positive,1
4998,No. I won`t be back until the 7th,neutral,0


In [14]:
y = tf.keras.utils.to_categorical(df["Labels"].values, num_classes=3)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25)

In [16]:
df['Labels'].value_counts()

 0    2022
 1    1592
-1    1385
Name: Labels, dtype: int64

In [17]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs= bert_encoder(preprocessed_text)

In [18]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = Dense(3, activation='softmax', name="output")(l)  # Connect the Dense layer to 'l'

model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [19]:
METRICS = [  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.Recall(name='F1')
]

In [20]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_word_ids': (None,                                           
                              128),                                                               
                              'input_type_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [22]:
model.fit(X_train, y_train,batch_size=64,
          epochs=10,
          verbose='auto',
          callbacks=None,
          validation_split=0.2,
          shuffle=True,
          class_weight=None,
          sample_weight=None,
          initial_epoch=0,
          steps_per_epoch=58,  # samples - (test_size * samples) / batch_size
          validation_steps=None,
          validation_batch_size=64,  # same as batch size
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False)

Epoch 1/10
58/58 [==============================] - 2336s 40s/step - loss: 1.1452 - accuracy: 0.6428 - precision: 0.3900 - recall: 0.1269 - F1: 0.1269 - val_loss: 1.0746 - val_accuracy: 0.6662 - val_precision: 0.4286 - val_recall: 0.0040 - val_F1: 0.0040
Epoch 2/10
58/58 [==============================] - 2369s 41s/step - loss: 1.0769 - accuracy: 0.6605 - precision: 0.4681 - recall: 0.1369 - F1: 0.1369 - val_loss: 1.0647 - val_accuracy: 0.6667 - val_precision: 0.5000 - val_recall: 0.0067 - val_F1: 0.0067
Epoch 3/10
58/58 [==============================] - 2374s 41s/step - loss: 1.0530 - accuracy: 0.6650 - precision: 0.4921 - recall: 0.1591 - F1: 0.1591 - val_loss: 1.0337 - val_accuracy: 0.6764 - val_precision: 0.5965 - val_recall: 0.0907 - val_F1: 0.0907
Epoch 4/10
58/58 [==============================] - 2327s 40s/step - loss: 1.0375 - accuracy: 0.6771 - precision: 0.5518 - recall: 0.1669 - F1: 0.1669 - val_loss: 1.0197 - val_accuracy: 0.6791 - val_precision: 0.5886 - val_recall: 0.12

58/58 [==============================] - 635s 11s/step - loss: 0.9990 - accuracy: 0.6756 - precision: 0.5260 - recall: 0.2693 - F1: 0.2693 - val_loss: 0.9849 - val_accuracy: 0.6991 - val_precision: 0.6714 - val_recall: 0.1907 - val_F1: 0.1907


In [27]:
model.evaluate(X_test,y_test)

40/40 [==============================] - 639s 16s/step - loss: 0.9746 - accuracy: 0.7048 - precision: 0.7173 - recall: 0.1888 - F1: 0.1888


[0.9745665192604065,
 0.704800009727478,
 0.7173252105712891,
 0.18880000710487366,
 0.18880000710487366]